# Image Based Visual Servoing



### Import camera calibration data

In [1]:
import pickle
import cv2
import numpy as np
import math
import time
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg

# Load COCO labels
filename = "coco_labels.dat"
filehandler = open(filename, 'rb')
COCO_labels = pickle.load(filehandler)

# Load camera calibration data for undistort
filename = "calibration.dat"
filehandler = open(filename, 'rb')
camera_cal = pickle.load(filehandler)

mtx = camera_cal['mtx']
dist = camera_cal['dist']

### Load pre-trained SSD model

In [2]:
from jetbot import ObjectDetector

model = ObjectDetector('../Notebooks/object_following/ssd_mobilenet_v2_coco.engine')

### Set up camera for object detection
* Camera is set up for 300x300 pixel video
* Use undistort() to undistort camera image prior to object detection

In [3]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

### Power up robot motor control

In [4]:
from jetbot import Robot

robot = Robot()

In [29]:
def undistort(img, mtx, dist, crop=False):
    """Undistort camera image based on calibration data"""
    h,  w = img.shape[:2]
    # print (h,w)
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
    
    # undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image if asked
    if crop:
        x,y,w,h = roi
        dst = dst[y:y+h, x:x+w]
    return dst

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

def create_feature_pts(bbox):
    """ Return 4 feature points of bounding box """
    # extract bounding x and y coordinates
    UL_fp, LR_fp = bbox
    UL_fp_x, UL_fp_y = UL_fp
    LR_fp_x, LR_fp_y = LR_fp
   
    # return UL, UR, LR, LL feature points
    return [(UL_fp_x, UL_fp_y), (LR_fp_x, UL_fp_y), (LR_fp_x, LR_fp_y), (UL_fp_x, LR_fp_y)]
    
def features_errors(currentFeatures, desiredFeatures):
    """ Return feature points error vector """
    UL_current, UR_current, LR_current, LL_current = currentFeatures 
    UL_desired, UR_desired, LR_desired, LL_desired = desiredFeatures 
    
    # calculate error for each feature point
    UL_error = fp_error(UL_current, UL_desired)
    error_vector = np.asarray(UL_error)
    UR_error = fp_error(UR_current, UR_desired)
    error_vector = np.concatenate((error_vector, UR_error), axis=None)
    LR_error = fp_error(LR_current, LR_desired)
    error_vector = np.concatenate((error_vector, LR_error), axis=None)
    LL_error = fp_error(LL_current, LL_desired)
    error_vector = np.concatenate((error_vector, LL_error), axis=None)

    # return 1x8 error vector of UL, UR, LR, LL feature points
    return np.reshape(error_vector, (1,-1))
    
    
def fp_error(current, desired):
    """ Return error of a feature point """
    current_x, current_y = current
    desired_x, desired_y = desired
    return (desired_x-current_x, desired_y-current_y)


def image_Jacobian(fp, mtx, depth):
    """ Generate image jacobiab L for a feature point """
    # focal lengths in pixel unit
    f_u = mtx[0,0]
    f_v = mtx[1,1]
    c_u = mtx[0,2]
    c_v = mtx[1,2]

    # Estimated distance of reference object (m)
    Z = depth

    # Calculate J of feature point
    u_raw, v_raw = fp
    u = u_raw - c_u
    v = v_raw - c_v    
    
    L = np.array([[-f_u/Z, 0, u/Z, u*v/f_u, -(f_u+u*u/f_u), v],
                  [0, -f_v/Z, v/Z, f_v+v*v/f_v, -u*v/f_v, -u]])
    return L

"""
def robot_jacobian(theta):
    
    return np.array([[math.cos(theta),0],
                  [math.sin(theta),0],
                  [0,0],
                  [0,0],
                  [0,0],
                  [0,1]])
"""

def robot_jacobian(robot_angle, wheel_radius, axle_length):
    """ calculate the robot jacobian """
    theta = robot_angle
    l = axle_length
    r = wheel_radius
    H = np.array([[math.cos(theta),0],
                  [math.sin(theta),0],
                  [0,0],
                  [0,0],
                  [0,0],
                  [0,1]])
    T = np.array([[r/2, r/2],
                  [r/l, -r/l]])
    
    return np.dot(H,T)

def control2motion(wheel_radius, axle_length):
    """ transform wheel speeds to robot motion in world frame """
    l = axle_length
    r = wheel_radius

    T = np.array([[r/2, r/2],
                  [r/l, -r/l]])
    
    return T

def robot2world(robot_angle):
    """ calculate the robot jacobian """
    theta = robot_angle
    H = np.array([[math.cos(theta),0],
                  [math.sin(theta),0],
                  [0,0],
                  [0,0],
                  [0,0],
                  [0,1]])
    return H

def speed_limit(motion):
    """ Clip wheel velocities """
    right_w = motion[0,0]
    left_w = motion[1,0]
    MOTION = 0.3
    
    max_abs = max(abs(right_w), abs(left_w))
    min_abs = min(abs(right_w), abs(left_w))
    
    if max_abs > 1.0:
        # Limit the peak forward velocity to 1.0
        w_r = right_w/max_abs
        w_l = left_w/max_abs
    elif max_abs < 0.1:
        # if velocities are below 0.1, set wheel speeds to zero
        w_r = 0
        w_l = 0
    else:
        if min_abs < 0.3:
            # Scale velocities above 0.3
            w_r = right_w*0.3/min_abs
            w_l = left_w*0.3/min_abs
        else:
            w_r = right_w
            w_l = left_w
        
    return w_l, w_r

### Open Image Widget

In [7]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)

width = int(image_widget.width)
height = int(image_widget.height)

In [68]:
start_bbox_TV = [(102, 90), (161, 134)]

pos1_bbox_TV = [(102, 90), (164, 135)]
pos1_bbox_stool = [(168, 123), (200, 182)]

pos2_bbox_TV = [(99, 79), (170, 128)]
pos2_bbox_stool = [(171, 115), (211, 183)]

pos2left_bbox_TV = [(136, 64), (218, 132)]
pos2left_bbox_stool = [(215, 96), (283, 187)]

pos2right_bbox_TV = [(51, 69), (134, 131)]
pos2right_bbox_stool = [(131, 122), (164, 184)]

pos3_bbox_TV = [(90, 72), (172, 127)]
pos3_bbox_stool = [(172, 107), (219, 189)]

beg_bbox_TV = [(72, 92), (136, 137)]
mid_bbox_TV = [(54, 70), (134, 121)]
mid_bbox_stool = [(175, 125), (216, 183)]
end_bbox_stool = [(199, 88), (270, 187)]

In [91]:
display(widgets.HBox([image_widget]))
depth = 2.0
robot_theta = -math.pi/6
wheel_radius = 0.0325
axle_length = 0.12
lamba = 10
Rtime = 0.1
"""
# Use stool as reference object
target = 62
end_target_bbox_pixel = end_bbox_stool
"""
# Use TV as reference object
target = 72
end_target_bbox_pixel = mid_bbox_TV

for i in range(1):

    undistort_image = undistort(camera.value, mtx, dist)
    detections = model(undistort_image)

     # draw all detections on image
    for det in detections[0]:
        label = COCO_labels[det['label']]
        bbox = det['bbox']
        bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                       (int(width * bbox[2]), int(height * bbox[3]))]
        cv2.rectangle(undistort_image, bbox_pixel[0], bbox_pixel[1], (255, 0, 0), 1)

        print(label,det['label'], bbox_pixel)

        # select detections that match selected class label
        matching_detections = [d for d in detections[0] if d['label'] == target]

        # get detection closest to center of field of view and draw it
        det = closest_detection(matching_detections)

        if det is not None:
            bbox = det['bbox']
            cv2.rectangle(undistort_image, (int(width * bbox[0]), int(width * bbox[1])),\
                                    (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 2)
            cv2.rectangle(undistort_image, end_target_bbox_pixel[0], end_target_bbox_pixel[1], \
                          (0, 0, 255), 2)
            target_bbox_pixel = [(int(width * bbox[0]), int(height * bbox[1])), 
                       (int(width * bbox[2]), int(height * bbox[3]))]

    image_widget.value = bgr8_to_jpeg(undistort_image)

    beg_feature_pts = create_feature_pts(target_bbox_pixel)
    end_feature_pts = create_feature_pts(end_target_bbox_pixel)
    print(target_bbox_pixel)
    print(end_target_bbox_pixel)

    error = features_errors(beg_feature_pts,end_feature_pts)
    print (error.T)

    UL_fp, UR_fp, LR_fp, LL_fp = beg_feature_pts

    # Generate image jacobian L
    L = np.vstack((image_Jacobian(UL_fp, mtx, depth), image_Jacobian(UR_fp, mtx, depth), \
              image_Jacobian(LR_fp, mtx, depth), image_Jacobian(LL_fp, mtx, depth)))

    J = robot_jacobian(robot_theta, wheel_radius, axle_length)

    wheel_motion = lamba*np.dot(np.linalg.pinv(np.dot(L,J)), error.T)
    print(wheel_motion)
    w_l, w_r = speed_limit(wheel_motion)
    print(w_l, w_r)

    Rtime = 0.1

    robot.set_motors(w_l, w_r)
    time.sleep(Rtime)
    robot.stop()

laptop 72 [(72, 90), (133, 135)]
couch 62 [(149, 130), (174, 181)]
[(72, 90), (133, 135)]
[(54, 70), (134, 121)]
[[-18]
 [-20]
 [  1]
 [-20]
 [  1]
 [-14]
 [-18]
 [-14]]
[[0.85879679]
 [2.8325448 ]]
1.0 0.3031891269029517


In [80]:
T = control2motion(wheel_radius, axle_length)
robot_motion = np.dot(T, wheel_motion)

print(robot_motion)

[[-0.05444397]
 [-0.00192063]]


In [85]:
bbox

[0.17554929852485657,
 0.21843597292900085,
 0.45806071162223816,
 0.4206964671611786]